<a href="https://colab.research.google.com/github/HighlandersFRC/2025-Robot/blob/Champs/Vision/Object_Detection/yolov8-to-rknn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv8 Training and Conversion to RKNN - Team 5990 TRIGON
<img src ="https://avatars.githubusercontent.com/u/45858082?s=280&v=4" />

In [1]:
#Run until it gets to training
%cd {root_path}
import os
root_path = os.getcwd()

!git clone https://github.com/airockchip/ultralytics_yolov8 ultralytics
%cd ultralytics
!git checkout 5b7ddd8f821c8f6edb389aa30cfbc88bd903867b

!pip install -e .

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.151 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.9/112.6 GB disk)


## Downloading the dataset
Input your Roboflow API key below. It can be obtained [here](https://app.roboflow.com/settings/api).
You can use your own dataset, the rest of the notebook should work with any number of classes, as long as the project is of "object detection" type.

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
import shutil

# Path to the file in Google Drive
drive_file_path = '/content/drive/My Drive/dataya.zip'

# Path to the destination in Colab
colab_file_path = '/content'

# Copy file from Google Drive to Colab environment
shutil.copy(drive_file_path, colab_file_path)

# Verify that the file is copied
print(f"File downloaded to Colab environment: {colab_file_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File downloaded to Colab environment: /content


In [4]:
import zipfile

# Path to your ZIP file in the Colab environment
zip_file_path = '/content/dataya.zip'

# Path to extract the files to
extracted_folder_path = '/content'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"Files extracted to {extracted_folder_path}")


Files extracted to /content


In [8]:
import os
import re
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
        print("Wrote successfully to " + line)

@register_line_cell_magic
def replaceAllInFile(line, cell):
    filename = line.strip()
    replacements = eval(cell)  # Assuming input is a valid Python expression
    with open(filename, 'r') as f:
        file_content = f.read()
    for replaced, with_this in replacements:
        file_content = re.sub(replaced, with_this, file_content)
    with open(filename, 'w') as f:
        f.write(file_content)
    print(f"Replaced successfully in {filename}")

In [13]:
%cat {dataset.location}/data.yaml

cat: {dataset.location}/data.yaml: No such file or directory


#### This is needed because the default format of the directories in roboflow datasets is kinda broken:

In [14]:
#New update fixes
import os

# Define paths
dataset_location = "/content/Reefscape FRC.v5i.yolov8"  # Update if your path is different
data_yaml_path = os.path.join(dataset_location, "data.yaml")

# Ensure the file exists before proceeding
if not os.path.exists(data_yaml_path):
    raise FileNotFoundError(f"File not found: {data_yaml_path}")

# Read the YAML file
with open(data_yaml_path, "r") as file:
    content = file.read()

# Define replacements
content = content.replace("test: ..", f"test: {dataset_location}/test")
content = content.replace("train: ", f"train: {dataset_location}/train")
content = content.replace("val: ", f"val: {dataset_location}/valid")  # Assuming "valid" is for validation

# Write the changes back to the file
with open(data_yaml_path, "w") as file:
    file.write(content)

print("data.yaml updated successfully!")


data.yaml updated successfully!


In [22]:
import yaml

# Load the YAML file and extract the number of classes
yaml_file_path = "/content/Reefscape FRC.v5i.yolov8/data.yaml"

with open(yaml_file_path, 'r') as stream:
    data = yaml.safe_load(stream)
    num_classes = data['nc']  # This should be an integer
    print(f"num_classes: {num_classes}")


num_classes: 3


In [27]:
%%writetemplate custom_yolov8.yaml
# Ultralytics YOLO 🚀, AGPL-3.0 license
# YOLOv8 object detection model with P3-P5 outputs. For Usage examples see https://docs.ultralytics.com/tasks/detect

# Parameters
nc: {num_classes}  # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024]  # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024]  # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768]   # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512]   # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512]   # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs
activation: nn.ReLU()
# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 12

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]  # cat head P4
  - [-1, 3, C2f, [512]]  # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]  # cat head P5
  - [-1, 3, C2f, [1024]]  # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]]  # Detect(P3, P4, P5)

Wrote successfully to custom_yolov8.yaml


## Training
You can adjust the following settings:
- model: one of [yolov8n, yolov8s, yolov8m, yolov8l, yolov8x]
- image_size: The input size of the images fed to the model. Should be a multiple of 32.
- batch: Number of samples per epoch. You should set this to the highest number possible without the training taking too much memory (it would crash if that happens, which is ok, just lower the number and try again)
- epochs: How many iterations to train for
- patience: After how many epochs without improvement to stop the training

In [ ]:
#Skip this whole section till you get to conversion
%cd {root_path}/ultralytics
model = "yolov8n"
image_size = 640
!yolo batch=192 epochs=700 patience=50 device=0 cache=ram task=detect mode=train model=./custom_{model}.yaml data={dataset.location}/data.yaml imgsz={image_size} plots=False pretrained=False single_cls={num_classes == '1'}

In [ ]:
latest_modified_time = 0
latest = None

for foldername, subfolders, filenames in os.walk(root_path):
    for filename in filenames:
        if filename == "best.pt":
            file_path = os.path.join(foldername, filename)
            modified_time = os.path.getmtime(file_path)
            if modified_time > latest_modified_time:
                latest_modified_time = modified_time
                latest = file_path
print(latest)

## Exporting to ONNX
This is an intermediate step between the PyTorch model and the RKNN model.

In [ ]:
%cd {root_path}/ultralytics
!yolo mode=export format=rknn model={latest}

In [ ]:
ex_path = '.'.join(latest.split('.')[:-1]) + '.onnx'
print(ex_path)

### Installing RKNN Toolkit 2

In [28]:
#Start again collab tends to mess things up here if you are not using it to train aka orin Follow comments to resolve all errors
!wget https://github.com/rockchip-linux/rknn-toolkit2/raw/2c2d03def0c0908c86985b8190e973976ecec74c/rknn-toolkit2/packages/rknn_toolkit2-1.6.0+81f21f4d-cp310-cp310-linux_x86_64.whl
!pip install ./rknn_toolkit2-1.6.0+81f21f4d-cp310-cp310-linux_x86_64.whl

--2025-03-28 01:19:36--  https://github.com/rockchip-linux/rknn-toolkit2/raw/2c2d03def0c0908c86985b8190e973976ecec74c/rknn-toolkit2/packages/rknn_toolkit2-1.6.0+81f21f4d-cp310-cp310-linux_x86_64.whl
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rockchip-linux/rknn-toolkit2/2c2d03def0c0908c86985b8190e973976ecec74c/rknn-toolkit2/packages/rknn_toolkit2-1.6.0%2B81f21f4d-cp310-cp310-linux_x86_64.whl [following]
--2025-03-28 01:19:36--  https://raw.githubusercontent.com/rockchip-linux/rknn-toolkit2/2c2d03def0c0908c86985b8190e973976ecec74c/rknn-toolkit2/packages/rknn_toolkit2-1.6.0%2B81f21f4d-cp310-cp310-linux_x86_64.whl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|

In [29]:
%cd {root_path}
!git clone https://github.com/airockchip/rknn_model_zoo/
%cd rknn_model_zoo
!git checkout eaa94d6f57ca553d493bf3bd7399a070452d2774
%cd examples/yolov8/python

/content
Cloning into 'rknn_model_zoo'...
remote: Enumerating objects: 3049, done.
remote: Counting objects: 100% (527/527), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 3049 (delta 418), reused 359 (delta 354), pack-reused 2522 (from 2)
Receiving objects: 100% (3049/3049), 278.53 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (1021/1021), done.
Updating files: 100% (1469/1469), done.
/content/rknn_model_zoo
Note: switching to 'eaa94d6f57ca553d493bf3bd7399a070452d2774'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.deta

In [30]:
%%writefile imgs.txt
imgs/1.jpg
imgs/2.jpg
imgs/3.jpg
imgs/4.jpg
imgs/5.jpg
imgs/6.jpg
imgs/7.jpg
imgs/8.jpg
imgs/9.jpg
imgs/10.jpg
imgs/11.jpg
imgs/12.jpg
imgs/13.jpg
imgs/14.jpg
imgs/15.jpg
imgs/16.jpg
imgs/17.jpg
imgs/18.jpg
imgs/19.jpg
imgs/20.jpg

Writing imgs.txt


In [32]:
import os
import shutil
import random
import glob

def copy_and_rename_images(source_folder, destination_folder, n):
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    image_files = glob.glob(os.path.join(source_folder, '*.jpg'))
    selected_images = random.sample(image_files, min(n, len(image_files)))
    for i, image_path in enumerate(selected_images, start=1):
        destination_path = os.path.join(destination_folder, f'{i}.jpg')
        shutil.copy(image_path, destination_path)
    print(f"{min(n, len(image_files))} random images copied from '{source_folder}' to '{destination_folder}' and renamed.")
copy_and_rename_images("/content/Reefscape FRC.v5i.yolov8"+"/train/images" , "imgs", 20)

20 random images copied from '/content/Reefscape FRC.v5i.yolov8/train/images' to 'imgs' and renamed.


In [33]:
%%replaceAllInFile {root_path}/rknn_model_zoo/examples/yolov8/python/convert.py
[
    ('../../../datasets/COCO/coco_subset_20.txt', 'imgs.txt'),
]

Replaced successfully in /content/rknn_model_zoo/examples/yolov8/python/convert.py


## Quantization
Here you choose whether to perform quantization, which makes the model lighter and faster, by converting all 32/16 bit floates in the model into 8 bit ints, which costs performance.

In [34]:
to_quantize = True # @param {type: "boolean"}

## Exporting to RKNN - Final step🎉

In [11]:
#May or may not need to run most likely don't

import os

# Define paths
root_path = "/content"  # Change this if needed
onnx_model_path = os.path.join(root_path, "best.onnx")
ex_path = os.path.join(root_path, "rknn_model_zoo/examples/yolov8/python")
output_dir = os.path.join(root_path, "rknn_model_zoo")  # Save inside rknn_model_zoo

# Set quantization type
to_quantize = True  # Change to False if you don't want quantization
quant_code = "i8" if to_quantize else "fp"

# Define output model path
image_size = 640  # Change this to your required input size
output_model = os.path.join(output_dir, f"best-{image_size}-{quant_code}.rknn")

# Run conversion script
os.chdir(ex_path)
os.system(f"python convert.py {onnx_model_path} rk3588 {quant_code} {output_model}")

print(f"RKNN model saved at: {output_model}")


RKNN model saved at: /content/rknn_model_zoo/best-640-i8.rknn


In [12]:

#Scripts for looking into the model zoo file for imgs.txt Collab sometimes messes this up
!ls -lh /content/rknn_model_zoo


total 4.1M
drwxr-xr-x  9 root root 4.0K Mar 28 01:20 3rdparty
-rw-r--r--  1 root root 4.0M Mar 28 01:30 best-640-i8.rknn
-rw-r--r--  1 root root 4.6K Mar 28 01:20 build-android.sh
-rw-r--r--  1 root root 4.6K Mar 28 01:20 build-linux.sh
drwxr-xr-x  6 root root 4.0K Mar 28 01:20 datasets
drwxr-xr-x 18 root root 4.0K Mar 28 01:20 examples
-rw-r--r--  1 root root  12K Mar 28 01:20 LICENSE
drwxr-xr-x  2 root root 4.0K Mar 28 01:20 py_utils
-rw-r--r--  1 root root  15K Mar 28 01:20 README_CN.md
-rw-r--r--  1 root root  16K Mar 28 01:20 README.md
-rw-r--r--  1 root root 9.8K Mar 28 01:20 scaling_frequency.sh
drwxr-xr-x  2 root root 4.0K Mar 28 01:20 utils


In [13]:
#Same as above
!find /content -name "*.rknn"



^C


In [38]:
!python /content/rknn_model_zoo/examples/yolov8/python/convert.py /content/best.onnx rk3588 i8 /content/rknn_model_zoo/best-640-i8.rknn


Traceback (most recent call last):
  File "/content/rknn_model_zoo/examples/yolov8/python/convert.py", line 4, in <module>
    from rknn.api import RKNN
ModuleNotFoundError: No module named 'rknn'


In [14]:
#Make sure you have this installed else the error from the script above will show
!pip install --upgrade rknn-toolkit2


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            

In [16]:
#if you have errors with imgs.txt this won't work else it will (Update this won't work just keep running scripts after)
!python /content/rknn_model_zoo/examples/yolov8/python/convert.py /content/best.onnx rk3588 i8 /content/rknn_model_zoo/best-640-i8.rknn


I rknn-toolkit2 version: 2.3.0
--> Config model
done
--> Loading model
I Loading : 100%|██████████████████████████████████████████████| 126/126 [00:00<00:00, 28543.47it/s]
done
--> Building model
I Quantizating : 100%|████████████████████████████████████████████| 161/161 [00:07<00:00, 20.28it/s]
W build: The default input dtype of 'images' is changed from 'float32' to 'int8' in rknn model for performance!
                       Please take care of this change when deploy rknn model with Runtime API!
W build: The default output dtype of '318' is changed from 'float32' to 'int8' in rknn model for performance!
                      Please take care of this change when deploy rknn model with Runtime API!
W build: The default output dtype of 'onnx::ReduceSum_326' is changed from 'float32' to 'int8' in rknn model for performance!
                      Please take care of this change when deploy rknn model with Runtime API!
W build: The default output dtype of '331' is changed from 'float32' 

In [2]:
#Make sure imgs.txt is where you want it to be
!ls -lh /content/rknn_model_zoo/examples/yolov8/python/imgs.txt


-rw-r--r-- 1 root root 231 Mar 28 01:20 /content/rknn_model_zoo/examples/yolov8/python/imgs.txt


In [3]:

!pwd


/content


In [4]:
import os
os.chdir("/content/rknn_model_zoo/examples/yolov8/python")


In [5]:
#Same thing
!ls -lh /content/rknn_model_zoo/examples/yolov8/python/imgs.txt


-rw-r--r-- 1 root root 231 Mar 28 01:20 /content/rknn_model_zoo/examples/yolov8/python/imgs.txt


In [10]:
#If Imgs.txt does not have any errors and it is in /content/rknn_model_zoo/examples/yolov8/python/imgs.textThis will work and train Make sure to rename model to have .onnx and put the right name here
!python /content/rknn_model_zoo/examples/yolov8/python/convert.py /content/best (2).onnx.onnx rk3588 i8 /content/best-640-i8.rknn


/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `python /content/rknn_model_zoo/examples/yolov8/python/convert.py /content/best (2).onnx.onnx rk3588 i8 /content/best-640-i8.rknn'
